## Description:

This project endeavors to create a sophisticated deep learning model with the capability to recognize food items accurately from images and estimate their calorie content. With the widespread use of smartphones and growing concern for health and nutrition, such a model could empower users to track their dietary intake more effectively, make informed food choices, and work towards their health and fitness objectives. Through the application of advanced machine learning techniques, including convolutional neural networks (CNNs) for image recognition and regression algorithms for calorie estimation, the model aims to offer users a convenient and accessible solution for managing their nutrition effectively.

In [51]:
#Load Data 
#==============================================
import tensorflow_datasets as tfds
#Tensorflow
#==============================================
import tensorflow as tf 

#Keras
#==============================================
from keras.applications.resnet import preprocess_input
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Dense,MaxPooling2D,Conv2D,Flatten
#Data Preprocessing 
#==============================================
from sklearn.model_selection import train_test_split 
#Warnings
#==============================================
import warnings
warnings.filterwarnings('ignore')

## Load The Data 

In [52]:
# Load training data
train_dataset, train_info = tfds.load('food101', split='train', with_info=True)

In [53]:
# Load validation data 
validation_dataset, validation_info = tfds.load('food101', split='validation', with_info=True)

## Resize The Image 

In [54]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label

In [55]:
dataset_preprocessed_train = train_dataset.map(lambda x: preprocess_image(x['image'], x['label']))
dataset_preprocessed_test=validation_dataset.map(lambda x:preprocess_image(x['image'],x['label']))

In [56]:
print(len(dataset_preprocessed_train))
print(len(dataset_preprocessed_test))

75750
25250


## Shape of The Images 

In [57]:
for image,label in dataset_preprocessed_train.take(1):
    image_shape=image.shape
    label_shspe=label.shape
    
print("Shape of a single preprocessed image:", image_shape)   
print("Shape of a single preprocessed label:", label_shspe) 

Shape of a single preprocessed image: (224, 224, 3)
Shape of a single preprocessed label: ()


## Model 

In [60]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(101, activation='softmax')  
])

In [66]:
batch_size = 32  # You can adjust the batch size as needed
batched_dataset_train = dataset_preprocessed_train.batch(batch_size)
batched_dataset_test = dataset_preprocessed_test.batch(batch_size)

In [67]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [68]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 26, 26, 128)      

In [69]:
history = model.fit(batched_dataset_train, epochs=10, validation_data=batched_dataset_test)

Epoch 1/10


2368/2368 [==============================] - 1723s 727ms/step - loss: 4.7395 - accuracy: 0.0085 - val_loss: 4.6152 - val_accuracy: 0.0099
Epoch 2/10
2368/2368 [==============================] - 1362s 575ms/step - loss: 4.6402 - accuracy: 0.0084 - val_loss: 4.6159 - val_accuracy: 0.0099
Epoch 3/10
2368/2368 [==============================] - 1358s 573ms/step - loss: 4.6261 - accuracy: 0.0086 - val_loss: 4.6152 - val_accuracy: 0.0099
Epoch 4/10
2368/2368 [==============================] - 1355s 572ms/step - loss: 4.6159 - accuracy: 0.0087 - val_loss: 4.6152 - val_accuracy: 0.0099
Epoch 5/10
2368/2368 [==============================] - 1358s 574ms/step - loss: 4.6159 - accuracy: 0.0085 - val_loss: 4.6152 - val_accuracy: 0.0099
Epoch 6/10
2368/2368 [==============================] - 1372s 579ms/step - loss: 4.6159 - accuracy: 0.0084 - val_loss: 4.6152 - val_accuracy: 0.0099
Epoch 7/10
2368/2368 [==============================] - 1365s 576ms/step - loss: 4.6159 - accuracy: 0.0085 - val_loss

### Calories For Each food

In [70]:
calorie_lookup = {
    'apple_pie': 300,
    'pizza': 800,
    'spaghetti_bolognese': 450,
    'hamburger': 600,
    'fried_rice': 350,
    'hot_dog': 300,
    'sushi': 400,
    'chocolate_cake': 500,
    'steak': 700,
    'grilled_salmon': 350,
    'caesar_salad': 200,
    'french_fries': 400,
    'omelette': 350,
    'pancakes': 350,
    'cheeseburger': 750,
    'donuts': 250,
    'lobster_bisque': 450,
    'club_sandwich': 600,
    'macaroni_and_cheese': 400,
    'pad_thai': 500,
    'chicken_curry': 450,
    'garlic_bread': 200,
    'chicken_wings': 500,
    'caprese_salad': 250,
    'tacos': 400,
    'guacamole': 200,
    'filet_mignon': 600,
    'nachos': 450,
    'ramen': 450,
    'hummus': 150,
    'spring_rolls': 200,
    'onion_rings': 350,
    'deviled_eggs': 250,
    'ceviche': 200,
    'beef_carpaccio': 300,
    'pulled_pork_sandwich': 550,
    'crab_cakes': 350,
    'french_onion_soup': 400,
    'clam_chowder': 350,
    'lobster_roll_sandwich': 450,
    'beef_tartare': 300,
    'escargots': 250,
    'chocolate_mousse': 350,
    'beet_salad': 200,
    'gnocchi': 400,
    'cannoli': 250,
    'bread_pudding': 350,
    'escabeche': 300,
    'fettuccine_alfredo': 600,
    'ceasar_salad': 250,
    'poutine': 700,
    'shrimp_and_grits': 400,
    'foie_gras': 900,
    'lobster': 600,
    'miso_soup': 100,
    'beef_tacos': 450,
    'beef_carpaccio': 300,
    'beef_tartare': 300,
    'beet_salad': 200,
    'beignets': 200,
    'bibimbap': 450,
    'bread_pudding': 350,
    'breakfast_burrito': 400,
    'bruschetta': 150,
    'cannoli': 250,
    'caprese_salad': 250,
    'carrot_cake': 300,
    'ceviche': 200,
    'cheese_plate': 300,
    'cheesecake': 350,
    'chicken_curry': 450,
    'chicken_quesadilla': 450,
    'chicken_wings': 500,
    'chocolate_cake': 500,
    'chocolate_mousse': 350,
    'churros': 300,
    'clam_chowder': 350,
    'club_sandwich': 600,
    'crab_cakes': 350,
    'creme_brulee': 300,
    'croque_madame': 450,
    'cup_cakes': 250,
    'deviled_eggs': 250,
    'donuts': 250,
    'dumplings': 300,
    'edamame': 150,
    'eggs_benedict': 500,
    'escargots': 250,
    'escabeche': 300,
    'filet_mignon': 600,
    'fish_and_chips': 600,
    'foie_gras': 900,
    'french_fries': 400,
    'french_onion_soup': 400,
    'french_toast': 400,
    'fried_calamari': 400,
    'fried_rice': 350,
    'garlic_bread': 200,
    'gnocchi': 400,
    'greek_salad': 250,
    'grilled_cheese_sandwich': 400,
    'grilled_salmon': 350,
    'guacamole': 200,
    'gyoza': 300,
    'hamburger': 600,
    'hot_and_sour_soup': 250,
    'hot_dog': 300,
    'huevos_rancheros': 500,
    'hummus': 150,
    'ice_cream': 200,
    'lasagna': 600,
    'lobster_bisque': 450,
    'lobster_roll_sandwich': 450,
    'macaroni_and_cheese': 400,
    'miso_soup': 100,
    'mussels': 400,
    'nachos': 450,
    'omelette': 350,
    'onion_rings': 350,
    'oysters': 200,
    'pad_thai': 500,
    'paella': 500,
    'pancakes': 350,
    'panna_cotta': 250,
    'peking_duck': 600,
    'pho': 450,
    'pizza': 800,
    'pork_chop': 600,
    'poutine': 700,
    'prime_rib': 800,
    'pulled_pork_sandwich': 550,
    'ramen': 450,
    'ravioli': 450,
    'red_velvet_cake': 350,
    'risotto': 450,
    'samosa': 300,
    'sashimi': 300,
    'scallops': 300,
    'seaweed_salad': 150,
    'shrimp_and_grits': 400,
    'spaghetti_bolognese': 450,
    'spaghetti_carbonara': 550,
    'spring_rolls': 200,
    'steak': 700,
    'strawberry_shortcake': 300,
    'sushi': 400,
    'tacos': 400,
    'takoyaki': 250,
    'tiramisu': 300,
    'tuna_tartare': 300,
    'waffles': 400
}


In [83]:
def estimate_calories(image, info):
    # Perform food recognition
    predicted_class = model.predict(image)
    class_label = info.features['label'].int2str(predicted_class.argmax())

    # Estimate calorie content
    if class_label in calorie_lookup:
        return calorie_lookup[class_label]
    else:
        return "Calorie information not available."


In [79]:
def load_image(file_path):
    # Read image from file
    img = tf.io.read_file(file_path)
    # Decode image
    img = tf.image.decode_image(img, channels=3)
    return img


In [84]:
# Load an image from file
image = load_image(r'C:\Users\Lenovo\tensorflow_datasets\food101\train\downloads\data.vision\food-101\images\apple_pie\21063.jpg')

# Preprocess the image
image_preprocessed, _ = preprocess_image(image, None)

# Reshape the preprocessed image to have a batch size of 1
image_preprocessed = tf.expand_dims(image_preprocessed, axis=0)

# Estimate calories
calories = estimate_calories(image_preprocessed, train_info)

print("Estimated Calories:", calories)

1/1 [==============================] - 0s 28ms/step
Estimated Calories: 350
